# RAG - Vector Search with Qdrant

```sh
docker run --rm --name qdrant\
     -p 6333:6333 \
     -p 6334:6334 \
     -v "${PWD}/qdrant_storage:/qdrant/storage:z" \
     qdrant/qdrant:v1.16.3
```

Go to http://localhost:6333/dashboard


Install packages


In [1]:
!uv pip install -q \
    requests==2.32.5 \
    python-dotenv==1.2.1 \
    litellm==1.78.5 \
    qdrant-client==1.16.2 \
    fastembed==0.7.4

Import packages


In [ ]:
import json
import random

import requests
from dotenv import load_dotenv
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

load_dotenv()

True

Download documents


In [ ]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]
    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

Create a Qdrant client instance


In [ ]:
client = QdrantClient("http://localhost:6333")

Verify models compatible with current setup


In [ ]:
EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

Choose model


In [ ]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

Define a collection


In [ ]:
collection_name = "zoomcamp-rag"

if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=EMBEDDING_DIMENSIONALITY, distance=models.Distance.COSINE
        ),
    )

Create points


In [ ]:
points = []
id = 0

for course in documents_raw:
    for doc in course["documents"]:
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc["text"], model=model_handle),
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            },
        )
        points.append(point)

        id += 1

Generate embeddings and insert into Qdrant


In [ ]:
client.upsert(collection_name=collection_name, points=points)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def search(query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(text=query, model=model_handle),
        limit=limit,
        with_payload=True,
    )

    return results

In [ ]:
course = random.choice(documents_raw)
course_piece = random.choice(course["documents"])
print(json.dumps(course_piece, indent=2))

{
  "text": "Check this article. If you know everything in this article, you know enough. If you don\u2019t, read the article and join the coursIntroduction to Pythone too :)\nIntroduction to Python \u2013 Machine Learning Bookcamp\nYou can follow this English course from the OpenClassrooms e-learning platform, which is free and covers the python basics for data analysis: Learn Python Basics for Data Analysis - OpenClassrooms . It is important to know some basics such as: how to run a Jupyter notebook, how to import libraries (and what libraries are), how to declare a variable (and what variables are) and some important operations regarding data analysis.\n(M\u00e9lanie Fouesnard)",
  "section": "General course-related questions",
  "question": "How much Python should I know?",
  "course": "machine-learning-zoomcamp"
}


In [ ]:
result = search(course_piece["question"])
result

QueryResponse(points=[ScoredPoint(id=445, version=1, score=0.84083724, payload={'text': 'Check this article. If you know everything in this article, you know enough. If you don’t, read the article and join the coursIntroduction to Pythone too :)\nIntroduction to Python – Machine Learning Bookcamp\nYou can follow this English course from the OpenClassrooms e-learning platform, which is free and covers the python basics for data analysis: Learn Python Basics for Data Analysis - OpenClassrooms . It is important to know some basics such as: how to run a Jupyter notebook, how to import libraries (and what libraries are), how to declare a variable (and what variables are) and some important operations regarding data analysis.\n(Mélanie Fouesnard)', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

Index fields that will be used as filters


In [ ]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword",
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

Search with filters


In [ ]:
def search_in_course(query, course, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(text=query, model=model_handle),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course", match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit,
        with_payload=True,
    )

    return results

Apply filters in search


In [ ]:
for course in [
    "data-engineering-zoomcamp",
    "machine-learning-zoomcamp",
    "mlops-zoomcamp",
]:
    print(
        course + "\n",
        search_in_course(
            "What if I submit homeworks late?",
            course=course,
        )
        .points[0]
        .payload["text"],
    )

data-engineering-zoomcamp
 No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]
machine-learning-zoomcamp
 Depends on whether the form will still be open. If you're lucky and it's open, you can submit your homework and it will be evaluated. if closed - it's too late.
(Added by Rileen Sinha, based on answer by Alexey on Slack)
mlops-zoomcamp
 Please choose the closest one to your answer. Also do not post your answer in the course slack channel.


## RAG
